In [ ]:
import pandas as pd
import numpy as np
import os

# 1. 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 기준 파일 로드
sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

# 3. 모델 로드 및 강제 정렬 함수
def load_and_align(filename):
    file_path = os.path.join(SUB_PATH, filename)
    df = pd.read_csv(file_path)
    
    # 행 개수 확인
    if len(df) != len(sample_sub):
        raise ValueError(f"[Error] {filename}의 행 개수({len(df)})가 sample_submission({len(sample_sub)})과 다릅니다.")
    
    # 만약 ID 컬럼이 있다면 정렬 안전장치 (없으면 순서 믿고 진행)
    if 'index' in df.columns:
         df = df.set_index('index').reindex(sample_sub['index']).reset_index()
    elif 'id' in df.columns: # 대소문자 고려
         df = df.set_index('id').reindex(sample_sub['id']).reset_index()
         
    return df['voted'].values

print(">>> 데이터 로딩 중...")
s1_voted = load_and_align('0130-1923.csv')               # m1 (Baseline)
s2_voted = load_and_align('Pure_0781_AUC_0.77322.csv')   # m2 (AUC Optimized)
s7_voted = load_and_align('m7_Refined_test_preds.csv')   # m7 (Deep FE)

# --------------------------------------------------------------------------------
# [Safety Check] 방향성(Correlation) 자동 검증 및 보정
# --------------------------------------------------------------------------------
corr_1_7 = np.corrcoef(s1_voted, s7_voted)[0, 1]
print(f"Checking Correlation (m1 vs m7): {corr_1_7:.4f}")

if corr_1_7 < 0:
    print("!!! 경고: m7과 m1의 상관관계가 음수입니다. m7의 방향을 반전합니다. !!!")
    # 랭크를 뒤집기 위해 값을 음수로 변환 (Rank 계산 시 큰 값이 작은 순위가 되도록 하거나, Rank 후 뒤집기)
    # 여기서는 직관적으로 Rank 계산 시 'ascending' 파라미터를 조절하기 위해 플래그 설정
    m7_ascending = False
else:
    print(">>> m7 방향성 정상 확인.")
    m7_ascending = True

# 4. Double-Rank Ensemble
print(">>> 랭크 앙상블 계산 중...")

# (1) 개별 랭크 변환 (pct=True로 0~1 사이 값으로 정규화)
# m1, m2는 검증되었으므로 기본적으로 정방향(작은 값이 1, 큰 값이 2 인지 확인 필요하지만 보통 그대로 씀)
# *중요: sample_submission의 타겟이 '투표 여부'일 때 보통 1(투표함), 2(투표안함) 이므로
# 모델 예측값이 확률(투표할 확률)이라면 값이 클수록 투표함(1)인가, 작을수록(1)인가를 따져야 함.
# 하지만 여기서는 'm1'을 기준으로 삼으므로 m1과 방향만 맞으면 됨.

r1 = pd.Series(s1_voted).rank(pct=True)
r2 = pd.Series(s2_voted).rank(pct=True)

if m7_ascending:
    r7 = pd.Series(s7_voted).rank(pct=True)
else:
    # 역방향인 경우: 값이 클수록 랭크가 낮아지게(혹은 그 반대) 뒤집음
    # 가장 간단한 건 1 - rank
    r7 = 1.0 - pd.Series(s7_voted).rank(pct=True)

# (2) 가중치 결합 (3:3:4)
avg_rank = (r1 * 0.3) + (r2 * 0.3) + (r7 * 0.4)

# (3) Scale Restoration (분포 복원) - argsort 방식이 더 안전함
# 원리: avg_rank가 가장 낮은 인덱스에 -> m1의 가장 낮은 값을 넣는다.
print(">>> m1 분포로 값 매핑(Restoration) 중...")

# m1의 값들을 정렬
m1_values_sorted = np.sort(s1_voted)

# 앙상블된 랭크의 순서를 구함 (argsort: 작은 값부터 순서대로 인덱스 반환)
ensemble_argsort = avg_rank.argsort().values

# 빈 배열 생성
final_voted = np.zeros_like(s1_voted)

# ensemble_argsort[0]은 앙상블 점수가 가장 낮은 사람의 인덱스 -> 여기에 m1 최솟값 매핑
# ensemble_argsort[1]은 앙상블 점수가 2등인 사람 -> 여기에 m1 2번째 작은 값 매핑
# ...
# 이를 한 번에 처리:
# 정렬된 인덱스 위치에 정렬된 값을 넣으면 안 되고,
# "원래 위치"에 "순위에 맞는 값"을 넣어야 함.

# 정확한 매핑을 위해 '순위(rank)'를 정수로 구함 (0 ~ N-1)
final_int_rank = avg_rank.rank(method='first').astype(int).values - 1

# final_voted[i] = m1_values_sorted[final_int_rank[i]]
# 예: i번째 사람의 순위가 0등이면, m1_sorted[0] 값을 가져감.
final_voted = m1_values_sorted[final_int_rank]

# 5. 최종 검증 및 저장
submission = sample_sub.copy()
submission['voted'] = final_voted

print("-" * 50)
print("검증 데이터 (m1 vs Final)")
print(f"Mean 오차: {abs(s1_voted.mean() - submission['voted'].mean()):.6f} (0에 가까워야 함)")
print(f"Std  오차: {abs(s1_voted.std() - submission['voted'].std()):.6f} (0에 가까워야 함)")
print(f"Min/Max 일치: {s1_voted.min() == submission['voted'].min()} / {s1_voted.max() == submission['voted'].max()}")
print("-" * 50)

output_name = os.path.join(SUB_PATH, '45_FINAL_TRINITY_FIXED_SAFE.csv')
submission.to_csv(output_name, index=False)
print(f"파일 생성 완료: {output_name}")

>>> 데이터 로딩 중...
Checking Correlation (m1 vs m7): -0.9910
!!! 경고: m7과 m1의 상관관계가 음수입니다. m7의 방향을 반전합니다. !!!
>>> 랭크 앙상블 계산 중...
>>> m1 분포로 값 매핑(Restoration) 중...
--------------------------------------------------
검증 데이터 (m1 vs Final)
Mean 오차: 0.000000 (0에 가까워야 함)
Std  오차: 0.000011 (0에 가까워야 함)
Min/Max 일치: True / True
--------------------------------------------------
파일 생성 완료: ./submissions/45_FINAL_TRINITY_FIXED_SAFE.csv
